안녕하세요, DS 26기 박하늘입니다.

이번 과제의 목표는 **Unsloth 라이브러리를 이용해
Mistral-7B-Instruct 모델을 4-bit quantization + LoRA 방식으로 빠르게 파인튜닝**하는 것입니다.

####1. 환경
- 실행 환경 : **Google Colab 무료 T4 GPU**
- RAM / VRAM 요구 : 약 15 GB 이하
- 필수 패키지 : `unsloth bitsandbytes accelerate trl datasets`

####2. 절차
1. 데이터 선택
  - HF Datasets Hub 에서 도메인 특화 데이터셋 1개 선택

  - 예) jtatman/medical-sci-instruct-1m-sharegpt

2. 템플릿·매핑 설정

  - chat_template="mistral"

  - mapping={"role":"from","content":"value","user":"human","assistant":"gpt"}

3. 전처리 & LoRA 파인튜닝

  - 샘플 수 : 5 k ~ 10 k (자유)

  - LoRA r/α : 16/16
 learning_rate=2e-4 , 60~300 step (기본 세팅은 이러하나, 자유)

4. 전·후 모델 비교

  - 동일 질문 3~5개에 대해 Base vs Fine-tuned 출력 비교

5. 결과 요약문 작성 (Results 참고)



# 설치

In [1]:
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # 코랩 환경에서 실행하는 경우
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.4

# [TODO 0, 1] 모델 설정

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 자유롭게 설정 가능
dtype = None # 자동 탐지의 경우 None. Tesla T4, V100 : Float16
load_in_4bit = True # 4 bit quantization 지원 여부, False로 설정 가능

# 4bit pre quantized 여러 모델을 지원하지만,,, mistral 7b를 사용 (다른 모델 사용하셔도 됩니다)
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
##TODO 0
##모델 내려받기

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # unsloth 모델이 아니어도 됩니다
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # unsloth quantized모델이 아닌 다른 gated model (예: Llama) 활용하는 경우 토큰 입력이 필요
)

# 파인튜닝 이후 모델과의 비교를 위해 학습 전 모델을 따로 복사
base_model, base_tok = FastLanguageModel.from_pretrained(
    model_name      = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length  = 2048,
    load_in_4bit    = True,
)
FastLanguageModel.for_inference(base_model)

==((====))==  Unsloth 2025.5.2: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

==((====))==  Unsloth 2025.5.2: Fast Mistral patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096, padding_idx=770)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): M

In [4]:
##TODO 1
##LoRA hyperparameter 설정

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # LoRA rank : 8, 16, 32, 64, 128 중에 선정
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", # LoRA를 삽입할 모듈 설정(projection layer)
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # LoRA 가중치 scaling : supports any, 일반적으로 r 혹은 2r로 설정
    lora_dropout = 0, # LoRA dropout rate : supports any, but = 0 is optimized
    bias = "none",    # LoRA bias param 추가 여부 : any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # supports rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# [TODO 2, 3] 데이터 준비

**데이터셋 필드 확인**
- 데이터셋에 따라 chat_template, mapping을 적절히 바꿔줘야 합니다.

**chat_template="..."**
- 일반적으로 모델이 학습된 템플릿 형식과 동일한 템플릿 이름을 지정 (예: mistral 모델의 경우 "mistral")
- 새 포맷으로 파인튜닝하고 싶다면, 학습과 추론 과정에서 같은 포맷 써야 함

**mapping={...}**
- 데이터셋의 JSON/CSV 안의 필드 명이 다른 경우 매핑 수정 필요
- 예시) 데이터 셋이 {"speaker":"client","utterance":"..."} 이런 형태인 경우 다음과 같이 수정
- mapping={"role":"speaker","content":"utterance","user":"client","assistant":"bot"}

**데이터셋 크기가 너무 크다면...**
- 다음과 같이 일부만 사용해서 학습
- dataset = load_dataset(dataset_name, split = "train[:5000]")


**참고자료 (unsloth chat_template)**
- 참고 : https://github.com/unslothai/unsloth/wiki#chat-templates

In [5]:
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

In [6]:
##TODO 2
##huggingface에서 원하는 데이터셋 불러오기
dataset_name="jtatman/medical-sci-instruct-1m-sharegpt"

##TODO 3
##각자 선택한 데이터셋에 맞도록 매핑, chat_template 수정
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "mistral", # 지원하는 형식: zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
    map_eos_token = True, # Maps <|im_end|> to </s> instead
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


dataset = load_dataset(dataset_name, split = "train[:10000]")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/547 [00:00<?, ?B/s]

data/train-00000-of-00003-1891745a93c8f9(…):   0%|          | 0.00/185M [00:00<?, ?B/s]

data/train-00001-of-00003-52adf7f9cca5d0(…):   0%|          | 0.00/186M [00:00<?, ?B/s]

data/train-00002-of-00003-c30cafa3a5ed3c(…):   0%|          | 0.00/186M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1255224 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
# 데이터 예시 출력 (매핑 전)
dataset[5]["conversations"]

[{'from': 'system',
  'value': "Let's think through this carefully, step by step, with attention to detail and verified results."},
 {'from': 'human',
  'value': 'Detailed Instructions: You will be given one or more triples. The second part of each triple shows the relation between the first and the third element. Your task is to write a simple and short piece of text (sentence(s)) that describes the triples in natural language.\nQ: Attica,_Indiana | isPartOf | Logan_Township,_Fountain_County,_Indiana\nA:'},
 {'from': 'gpt',
  'value': 'Attica, Indiana is part of Logan Township, Fountain County, Indiana.'}]

In [8]:
# 데이터 예시 출력 (매핑 후)
print(dataset[5]["text"])

<s>[INST] Let's think through this carefully, step by step, with attention to detail and verified results. Detailed Instructions: You will be given one or more triples. The second part of each triple shows the relation between the first and the third element. Your task is to write a simple and short piece of text (sentence(s)) that describes the triples in natural language.
Q: Attica,_Indiana | isPartOf | Logan_Township,_Fountain_County,_Indiana
A: [/INST]Attica, Indiana is part of Logan Township, Fountain County, Indiana.</s>


#### (If required) 커스텀 chat template

In [9]:
# 커스텀 chat template을 만드는 것도 가능함
unsloth_template = \
    "{{ bos_token }}"\
    "{{ 'You are a helpful assistant to the user\n' }}"\
    "{% for message in messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ '>>> User: ' + message['content'] + '\n' }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ '>>> Assistant: ' + message['content'] + eos_token + '\n' }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}"\
        "{{ '>>> Assistant: ' }}"\
    "{% endif %}"
unsloth_eos_token = "eos_token"

if False:
    tokenizer = get_chat_template(
        tokenizer,
        chat_template = (unsloth_template, unsloth_eos_token,),
        mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
        map_eos_token = True, # Maps <|im_end|> to </s> instead
    )

# [TODO 4] Training

- huggingface Trl의 SFTTrainer 활용해서 training 진행
-[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

##TODO 4
##모델 학습 hyperparameter 설정
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", #
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 10,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/7,000,000,000 (0.60% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.931000
2,3.321500
3,2.121200
4,2.645800
5,2.462700
6,1.907900
7,2.347800
8,2.184800
9,1.468300
10,1.683500


# Inference


In [12]:
# 1) 두 모델 모두 inference 모드 최적화
FastLanguageModel.for_inference(base_model)   # 원본
FastLanguageModel.for_inference(model)        # 파인튜닝 모델

# 2) 비교할 질문 리스트
questions = [
    "Explain what an MRI scan is in simple terms.",
    "What are the symptoms of type-2 diabetes?",
    "Give me a one-sentence summary of the placebo effect.",
]

def ask_any(model, prompt, max_new_tokens=128):
    """단일 질문 → 모델 답변 문자열 반환"""
    msgs = [{"from": "human", "value": prompt}]
    inputs = tokenizer.apply_chat_template(
        msgs,
        tokenize=True,
        add_generation_prompt=True,   # assistant 쪽을 모델이 채우도록 설정
        return_tensors="pt",
        padding=False
    ).to("cuda")                      # 모델들이 GPU에 있으므로
    with torch.no_grad():
        out = model.generate(
            input_ids=inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
        )
    return tokenizer.decode(out[0], skip_special_tokens=True).strip()

# 3) 전-후 결과 프린트
for q in questions:
    print(f"\n🟡  **Q**: {q}\n")
    print("🔹  ▸  _Base-Model_:")
    print(ask_any(base_model, q))
    print("\n🔸  ▸  _Fine-tuned_:")
    print(ask_any(model, q))
    print("-" * 80)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



🟡  **Q**: Explain what an MRI scan is in simple terms.

🔹  ▸  _Base-Model_:
Explain what an MRI scan is in simple terms.  An MRI (Magnetic Resonance Imaging) scan is a type of medical imaging test that uses a strong magnetic field and radio waves to create detailed images of the organs and tissues within your body.

Here's a simple breakdown:

1. You lie on a table that slides into a large, tube-shaped machine. This machine creates a strong magnetic field around you.

2. Radio waves are then sent into your body, which causes the atoms in your body to align with the magnetic field.

3. When the radio waves are turned off, the aligned atoms emit a

🔸  ▸  _Fine-tuned_:
Explain what an MRI scan is in simple terms. An MRI scan is a type of medical imaging test that uses strong magnetic fields and radio waves to create detailed images of the body's internal structures. It is often used to diagnose or monitor conditions such as tumors, injuries, or diseases of the brain or spinal cord.
-----

# [TODO 5] Results

## 데이터 요약
- 데이터셋 이름:
- 데이터셋 도메인(의료, 법률 등):
- 데이터셋 중 학습에 활용한 샘플 수:
- 템플릿, 매핑 설정 간단 요약:

## 파인튜닝
- 하이퍼 파라미터 변경사항:
- 학습 시간:
- training loss 마지막 값:

## 전후 예시 비교
- 예시 추론으로 본 Base 모델과 FT 모델 차이점:

## (additional) 모델 저장과 로드

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [13]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [14]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "What is a famous tall tower in Paris?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

<s>[INST] What is a famous tall tower in Paris? [/INST]The Eiffel Tower is a famous tall tower in Paris.</s>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [15]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoModelForPeftCausalLM
    from transformers import AutoTokenizer

    model = AutoModelForPeftCausalLM.from_pretrained(
        "lora_model",  # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit=load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [16]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [17]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")